# Welcome Notebook
This notebook walks through the process of creating and populating your first database with FinSpace Managed KX.

## Before you start
Before you start this notebook, it is assumed you have the following:
- FinSpace Managed KX environment created in AWS account
- S3 staging bucket for data and code
  - This notebook boto's profile and the Managed KX environment can access the bucket
- Setup in ~/.aws directory
  - config is set (json and region)
  - default credentials are set (aws_access_key_id, aws_secret_access_key, aws_session_token)

## Steps
1. Untar hdb.tar.gz for the hdb data
2. Upload hdb to staging S3 bucket
3. Create database
4. Add HDB data to database
5. Create a Cluster
6. Get the connectionString

## Setup
imports, environmentId, accountId

In [1]:
import os
import boto3
import json
import datetime

from managed_kx import *
from env2 import *


In [2]:
# Source data directory
SOURCE_DATA_DIR="hdb"

# S3 bucket for external data and code
S3_DEST=f"s3://{S3_BUCKET}/data/{SOURCE_DATA_DIR}/"
CODEBASE="code"
CODE_PATH=f"code/{CODEBASE}.zip"

# Managed KX Database and Cluster names to create
DB_NAME="welcomedb"
DELETE_CLUSTER=False
DELETE_DATABASE=False

create_delete=True

if create_delete:
    DB_NAME="create_delete_db"
    DELETE_CLUSTER=True
    DELETE_DATABASE=True
    
CLUSTER_NAME=f"cluster_{DB_NAME}"


In [3]:
# triggers credential get
session=None

try:
    subprocess.call(["which", "ada"])
    os.system(f"ada credentials update --account={ACCOUNT_ID} --provider=isengard --role=Admin --once")
except: 
    None

if AWS_ACCESS_KEY_ID is None:
    print("Using Defaults ...")
    # create AWS session: using access variables
    session = boto3.Session()
else:
    print("Using variables ...")
    session = boto3.Session(
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        aws_session_token=AWS_SESSION_TOKEN
    )

# create finspace client
client = session.client(service_name='finspace', endpoint_url=ENDPOINT_URL)

Using variables ...


# 0. Environment Check
Be sure the infrastructure ID has been entitled to the bucket you will be staging the HDB to. The environment will also need access to the KMX key used when creating the environment.

## Permission Templates

### S3 Permission
Example of code and data access to the same S3 bucket.

```
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "finspace.amazonaws.com"
            },
            "Action": [
                "s3:PutObject",
                "s3:PutObjectTagging",
                "s3:PutObjectAcl",
                "s3:GetObject",
                "s3:GetObjectVersion",
                "s3:GetObjectTagging",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::S3_BUCKET/*",
                "arn:aws:s3:::S3_BUCKET"
            ],
            "Condition": {
                "StringEquals": {
                    "aws:SourceAccount": "ACCOUNT_ID"
                },
                "ArnEquals": {
                    "aws:SourceArn": "arn:aws:finspace:us-east-1:ACCOUNT_ID:kxEnvironment/ENV_ID/*"
                }
            }
        }
    ]
}

```

### KMS Key
Be sure the environment has access to use the KMS key given in environment creation.

```
"Statement": [
        {
            "Sid": "Enable Managed kdb Insights Access",
            "Effect": "Allow",
            "Principal": {
                "Service": "finspace.amazonaws.com"
            },
            "Action": [
                "kms:Encrypt",
                "kms:Decrypt",
                "kms:GenerateDataKey"
            ],
            "Resource": "arn:aws:kms:us-east-1:ACCOUNT_ID:key/KEY_ID",
            "Condition": {
                "StringEquals": {
                    "aws:SourceAccount": "ACCOUNT_ID"
                },
                "ArnLike": {
                    "aws:SourceArn": "arn:aws:finspace:us-east-1:ACCOUNT_ID:kxEnvironment/ENV_ID/*"
                }
            }
        }
   ]
```


In [4]:
resp=get_kx_environment(client, environmentId=ENV_ID)

print("Environment Information")
print(json.dumps(resp,sort_keys=True,indent=4,default=str))


Environment Information
{
    "availabilityZoneIds": [
        "use1-az6",
        "use1-az4",
        "use1-az1"
    ],
    "awsAccountId": "612841383594",
    "certificateAuthorityArn": "arn:aws:acm-pca:us-east-1:952604696585:certificate-authority/0bd40a67-e61c-4e0f-8d83-9f84b10b4653",
    "creationTimestamp": "2023-05-30T21:49:29.29Z",
    "dedicatedServiceAccountId": "952604696585",
    "description": "Anvironment created after last beta release ahead of GA",
    "dnsStatus": "NONE",
    "environmentArn": "arn:aws:finspace:us-east-1:612841383594:kxEnvironment/itcdoumzc5cixt5vh4t6dp",
    "environmentId": "itcdoumzc5cixt5vh4t6dp",
    "kmsKeyId": "arn:aws:kms:us-east-1:612841383594:key/bbfad1fa-9e38-47f1-986d-33fb976a9ec4",
    "name": "Managed_KX_Beta_20230530",
    "status": "CREATED",
    "tgwStatus": "SUCCESSFULLY_UPDATED",
    "transitGatewayConfiguration": {
        "routableCIDRSpace": "100.64.0.0/26",
        "transitGatewayID": "tgw-08a5795d33483b14f"
    },
    "updateTime

## 1. Untar hdb.tar.gz
hdb database will be found in hdb directory

In [5]:
!tar -xf hdb.tar.gz

In [6]:
!ls -la hdb

total 52
drwxr-xr-x. 12 ec2-user ec2-user 16384 Apr 24 23:17 .
drwxr-xr-x.  7 ec2-user ec2-user 16384 May 31 14:59 ..
drwxr-xr-x.  3 ec2-user ec2-user    21 Apr 24 23:17 2023.04.14
drwxr-xr-x.  3 ec2-user ec2-user    21 Apr 24 23:17 2023.04.15
drwxr-xr-x.  3 ec2-user ec2-user    21 Apr 24 23:17 2023.04.16
drwxr-xr-x.  3 ec2-user ec2-user    21 Apr 24 23:17 2023.04.17
drwxr-xr-x.  3 ec2-user ec2-user    21 Apr 24 23:17 2023.04.18
drwxr-xr-x.  3 ec2-user ec2-user    21 Apr 24 23:17 2023.04.19
drwxr-xr-x.  3 ec2-user ec2-user    21 Apr 24 23:17 2023.04.20
drwxr-xr-x.  3 ec2-user ec2-user    21 Apr 24 23:17 2023.04.21
drwxr-xr-x.  3 ec2-user ec2-user    21 Apr 24 23:17 2023.04.22
drwxr-xr-x.  3 ec2-user ec2-user    21 Apr 24 23:17 2023.04.23
-rw-r--r--.  1 ec2-user ec2-user 16392 Apr 24 23:17 sym


# 2. Upload hdb data
using aws cli, copy hdb to staging bucket

In [7]:
if AWS_ACCESS_KEY_ID is not None:
    cp = f"""
export AWS_ACCESS_KEY_ID={AWS_ACCESS_KEY_ID}
export AWS_SECRET_ACCESS_KEY={AWS_SECRET_ACCESS_KEY}
export AWS_SESSION_TOKEN={AWS_SESSION_TOKEN}

aws s3 sync  --exclude .DS_Store {SOURCE_DATA_DIR} {S3_DEST}
aws s3 ls {S3_DEST}
"""
else:
    cp = f"""
aws s3 sync  --exclude .DS_Store {SOURCE_DATA_DIR} {S3_DEST}
aws s3 ls {S3_DEST}
"""
    
# execute the S3 copy
os.system(cp)

                           PRE 2023.01.29/
                           PRE 2023.01.30/
                           PRE 2023.01.31/
                           PRE 2023.02.01/
                           PRE 2023.02.02/
                           PRE 2023.02.03/
                           PRE 2023.02.04/
                           PRE 2023.02.05/
                           PRE 2023.02.06/
                           PRE 2023.02.07/
                           PRE 2023.04.14/
                           PRE 2023.04.15/
                           PRE 2023.04.16/
                           PRE 2023.04.17/
                           PRE 2023.04.18/
                           PRE 2023.04.19/
                           PRE 2023.04.20/
                           PRE 2023.04.21/
                           PRE 2023.04.22/
                           PRE 2023.04.23/
2023-04-27 17:42:55      16392 sym


0

## 3. Create database

In [8]:
# assume it exists
create_db=False

try:
    resp = client.get_kx_database(environmentId=ENV_ID, databaseName=DB_NAME)
    resp.pop('ResponseMetadata', None)
except:
    # does not exist, will create
    create_db=True

if create_db:
    print(f"CREATING Database: {DB_NAME}")
    resp = client.create_kx_database(environmentId=ENV_ID, databaseName=DB_NAME, description="Welcome kdb database")
    resp.pop('ResponseMetadata', None)

    print(f"CREATED Database: {DB_NAME}")

print(json.dumps(resp,sort_keys=True,indent=4,default=str))

CREATING Database: create_delete_db
CREATED Database: create_delete_db
{
    "createdTimestamp": "2023-05-31 15:23:00.011000+00:00",
    "databaseArn": "arn:aws:finspace:us-east-1:612841383594:kxEnvironment/itcdoumzc5cixt5vh4t6dp/kxDatabase/create_delete_db",
    "databaseName": "create_delete_db",
    "description": "Welcome kdb database",
    "environmentId": "itcdoumzc5cixt5vh4t6dp",
    "lastModifiedTimestamp": "2023-05-31 15:23:00.011000+00:00"
}


## 4. Add HDB data to database

In [9]:
changes=[]

for f in os.listdir("hdb"):
    if os.path.isdir(f"hdb/{f}"):
        changes.append( { 'changeType': 'PUT', 's3Path': f"{S3_DEST}{f}/", 'dbPath': f"/{f}/" } )
    else:
        changes.append( { 'changeType': 'PUT', 's3Path': f"{S3_DEST}{f}", 'dbPath': f"/" } )
        
resp = client.create_kx_changeset(environmentId=ENV_ID, databaseName=DB_NAME, 
    changeRequests=changes)

resp.pop('ResponseMetadata', None)
changeset_id = resp['changesetId']

print("Changeset...")
print(json.dumps(resp,sort_keys=True,indent=4,default=str))

Changeset...
{
    "changeRequests": [
        {
            "changeType": "PUT",
            "dbPath": "/2023.04.23/",
            "s3Path": "s3://kdb-demo-612841383594/data/hdb/2023.04.23/"
        },
        {
            "changeType": "PUT",
            "dbPath": "/2023.04.15/",
            "s3Path": "s3://kdb-demo-612841383594/data/hdb/2023.04.15/"
        },
        {
            "changeType": "PUT",
            "dbPath": "/2023.04.14/",
            "s3Path": "s3://kdb-demo-612841383594/data/hdb/2023.04.14/"
        },
        {
            "changeType": "PUT",
            "dbPath": "/2023.04.22/",
            "s3Path": "s3://kdb-demo-612841383594/data/hdb/2023.04.22/"
        },
        {
            "changeType": "PUT",
            "dbPath": "/2023.04.18/",
            "s3Path": "s3://kdb-demo-612841383594/data/hdb/2023.04.18/"
        },
        {
            "changeType": "PUT",
            "dbPath": "/2023.04.20/",
            "s3Path": "s3://kdb-demo-612841383594/data/hdb/2

In [10]:
wait_for_changeset_status(client, ENV_ID, DB_NAME, changeset_id, show_wait=True)

Status is IN_PROGRESS, total wait 0:00:00, waiting 10 sec ...
Status is IN_PROGRESS, total wait 0:00:10, waiting 10 sec ...
Status is IN_PROGRESS, total wait 0:00:20, waiting 10 sec ...
Status is IN_PROGRESS, total wait 0:00:30, waiting 10 sec ...


{'changesetId': 'nsQ5M93T4nKfI0KaAWPH3w',
 'databaseName': 'create_delete_db',
 'environmentId': 'itcdoumzc5cixt5vh4t6dp',
 'changeRequests': [{'changeType': 'PUT',
   's3Path': 's3://kdb-demo-612841383594/data/hdb/2023.04.23/',
   'dbPath': '/2023.04.23/'},
  {'changeType': 'PUT',
   's3Path': 's3://kdb-demo-612841383594/data/hdb/2023.04.15/',
   'dbPath': '/2023.04.15/'},
  {'changeType': 'PUT',
   's3Path': 's3://kdb-demo-612841383594/data/hdb/2023.04.14/',
   'dbPath': '/2023.04.14/'},
  {'changeType': 'PUT',
   's3Path': 's3://kdb-demo-612841383594/data/hdb/2023.04.22/',
   'dbPath': '/2023.04.22/'},
  {'changeType': 'PUT',
   's3Path': 's3://kdb-demo-612841383594/data/hdb/2023.04.18/',
   'dbPath': '/2023.04.18/'},
  {'changeType': 'PUT',
   's3Path': 's3://kdb-demo-612841383594/data/hdb/2023.04.20/',
   'dbPath': '/2023.04.20/'},
  {'changeType': 'PUT',
   's3Path': 's3://kdb-demo-612841383594/data/hdb/2023.04.16/',
   'dbPath': '/2023.04.16/'},
  {'changeType': 'PUT',
   's3Pat

In [11]:
note_str = ""

c_set_list = list_kx_changesets(client, ENV_ID, DB_NAME)

if len(c_set_list) == 0:
    note_str = "<<Could not get changesets>>"
    
print(100*"=")
print(f"Database: {DB_NAME}, Changesets: {len(c_set_list)} {note_str}")
print(100*"=")

# sort by create time
c_set_list = sorted(c_set_list, key=lambda d: d['createdTimestamp']) 

for c in c_set_list:
    c_set_id = c['changesetId']
    print(f"  Changeset: {c_set_id}: Created: {c['createdTimestamp']} ({c['status']})")
    c_rqs = client.get_kx_changeset(environmentId=ENV_ID, databaseName=DB_NAME, changesetId=c_set_id)['changeRequests']

    chs_pdf = pd.DataFrame.from_dict(c_rqs).style.hide(axis='index')
    display(chs_pdf)

Database: create_delete_db, Changesets: 1 
  Changeset: nsQ5M93T4nKfI0KaAWPH3w: Created: 2023-05-31 15:23:00.905000+00:00 (COMPLETED)


changeType,s3Path,dbPath
PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.23/,/2023.04.23/
PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.15/,/2023.04.15/
PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.14/,/2023.04.14/
PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.22/,/2023.04.22/
PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.18/,/2023.04.18/
PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.20/,/2023.04.20/
PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.16/,/2023.04.16/
PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.17/,/2023.04.17/
PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.21/,/2023.04.21/
PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.19/,/2023.04.19/


## 5. Create a Cluster for the database

In [12]:
# zip the code
os.system(f"zip -r -X {CODEBASE}.zip {CODEBASE} -x '*.ipynb_checkpoints*'")

# copy code to S3

if AWS_ACCESS_KEY_ID is not None:
    cp = f"""
export AWS_ACCESS_KEY_ID={AWS_ACCESS_KEY_ID}
export AWS_SECRET_ACCESS_KEY={AWS_SECRET_ACCESS_KEY}
export AWS_SESSION_TOKEN={AWS_SESSION_TOKEN}

aws s3 cp  --exclude .DS_Store {CODEBASE}.zip s3://{S3_BUCKET}/code/{CODEBASE}.zip
aws s3 ls s3://{S3_BUCKET}/code/
"""
else:
    cp = f"""
aws s3 cp  --exclude .DS_Store {CODEBASE}.zip s3://{S3_BUCKET}/code/{CODEBASE}.zip
aws s3 ls s3://{S3_BUCKET}/code/
"""
    
# execute the S3 copy
os.system(cp)

updating: code/ (stored 0%)
updating: code/lib.q (stored 0%)
updating: code/init.q (deflated 54%)
upload: ./code.zip to s3://kdb-demo-612841383594/code/code.zip     
2023-05-31 00:15:34      28450 basictick.zip
2023-05-31 15:23:45        757 code.zip
2023-05-31 10:19:36        652 taqcode.zip
2023-05-31 10:24:55        787 welcomedb.zip


0

In [13]:
print(f"Creating: {CLUSTER_NAME}")

resp = client.create_kx_cluster(
    environmentId=ENV_ID, 
    clusterName=CLUSTER_NAME,
    clusterDescription=f"Demo Cluster for database {DB_NAME}",
    clusterType='HDB',
    releaseLabel = '1.0',
    capacityConfiguration={ "nodeType": "kx.s.xlarge", "nodeCount": 3 },
    databases=[{ 
        'databaseName': DB_NAME, 
        'cacheConfigurations': [
            {'dbPaths':['/'], 'cacheType': 'CACHE_1000' }
        ] 
    }],
    cacheStorageConfigurations=[{ 'type': 'CACHE_1000', 'size':1200 }],
    azMode=AZ_MODE,
    availabilityZoneId=AZ_ID,
    vpcConfiguration={ 
        'vpcId': VPC_ID,
        'securityGroupIds': SECURITY_GROUPS,
        'subnetIds': SUBNET_IDS,
        'ipAddressType': 'IP_V4' },
    code={ 's3Bucket': S3_BUCKET, 's3Key': CODE_PATH },
    initializationScript=f"{CODEBASE}/init.q",
    commandLineArguments=[
        {'key': 's', 'value': '4'}, 
        {'key': 'dbname', 'value': DB_NAME}, 
        {'key': 'codebase', 'value': CODEBASE}
    ]
)

Creating: cluster_create_delete_db


In [14]:
resp

{'ResponseMetadata': {'RequestId': '71b76ba7-68d0-457a-94f2-24122f15c87a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json',
   'content-length': '1128',
   'connection': 'keep-alive',
   'date': 'Wed, 31 May 2023 15:23:53 GMT',
   'x-amzn-requestid': '71b76ba7-68d0-457a-94f2-24122f15c87a',
   'x-amz-apigw-id': 'Fyz0RGEioAMFm-w=',
   'x-amzn-trace-id': 'Root=1-64776681-09e6a76e17ece19b2b85f2af',
   'x-cache': 'Miss from cloudfront',
   'via': '1.1 b2179245b8d8ae2b245dd8946895eb1e.cloudfront.net (CloudFront)',
   'x-amz-cf-pop': 'IAD55-P1',
   'x-amz-cf-id': 'gAQIQeb33bwr4apE-R8xSqZrmzScu0AjztPJn0tMby_zQN9ctZvUAQ=='},
  'RetryAttempts': 0},
 'status': 'PENDING',
 'clusterName': 'cluster_create_delete_db',
 'clusterType': 'HDB',
 'databases': [{'databaseName': 'create_delete_db',
   'cacheConfigurations': [{'cacheType': 'CACHE_1000', 'dbPaths': ['/']}],
   'changesetId': 'nsQ5M93T4nKfI0KaAWPH3w'}],
 'cacheStorageConfigurations': [{'type': 'CACHE_1000', 'size'

In [15]:
wait_for_cluster_status(client, environmentId=ENV_ID, clusterName=CLUSTER_NAME, show_wait=True)
print()
print("** DONE **")

Cluster: cluster_create_delete_db status is PENDING, total wait 0:00:00, waiting 30 sec ...
Cluster: cluster_create_delete_db status is CREATING, total wait 0:00:30, waiting 30 sec ...
Cluster: cluster_create_delete_db status is CREATING, total wait 0:01:00, waiting 30 sec ...
Cluster: cluster_create_delete_db status is CREATING, total wait 0:01:30, waiting 30 sec ...
Cluster: cluster_create_delete_db status is CREATING, total wait 0:02:00, waiting 30 sec ...
Cluster: cluster_create_delete_db status is CREATING, total wait 0:02:30, waiting 30 sec ...
Cluster: cluster_create_delete_db status is CREATING, total wait 0:03:00, waiting 30 sec ...
Cluster: cluster_create_delete_db status is CREATING, total wait 0:03:30, waiting 30 sec ...
Cluster: cluster_create_delete_db status is CREATING, total wait 0:04:00, waiting 30 sec ...
Cluster: cluster_create_delete_db status is CREATING, total wait 0:04:30, waiting 30 sec ...
Cluster: cluster_create_delete_db status is CREATING, total wait 0:05:0

## 6. Get the connectionString
This assumes that the IAM role exists and the user (KDB_USERNAME) have beed already added as well.

In [16]:
try:
    resp = client.get_kx_cluster(environmentId=ENV_ID, clusterName=CLUSTER_NAME)
except client.exceptions.ResourceNotFoundException:
    print(F"Cluster: {CLUSTER_NAME} did not create")
    
if resp['ResponseMetadata']['HTTPStatusCode'] != 200:
    sys.stderr.write("Error:\n {resp}")
else:
    resp.pop('ResponseMetadata', None)

kx_cluster = resp

print("Cluster: "+("-"*80))
print(json.dumps(kx_cluster, sort_keys=True, indent=4, default=str))

# Give permissions time to propogate after cluster creation....
time.sleep(25)

conn_str = get_kx_connection_string(client, environmentId=ENV_ID, clusterName=CLUSTER_NAME, userName=KDB_USERNAME, boto_session=session)

print ("")
print("Copy into q: "+("-"*80))
print(f"""
/ Cluster: {CLUSTER_NAME}
hdb_conn:"{conn_str}"
""")

Cluster: --------------------------------------------------------------------------------
{
    "availabilityZoneId": "use1-az4",
    "azMode": "SINGLE",
    "cacheStorageConfigurations": [
        {
            "size": 1200,
            "type": "CACHE_1000"
        }
    ],
    "capacityConfiguration": {
        "nodeCount": 3,
        "nodeType": "kx.s.xlarge"
    },
    "clusterDescription": "Demo Cluster for database create_delete_db",
    "clusterName": "cluster_create_delete_db",
    "clusterType": "HDB",
    "code": {
        "s3Bucket": "kdb-demo-612841383594",
        "s3Key": "code/code.zip"
    },
    "commandLineArguments": [
        {
            "key": "s",
            "value": "4"
        },
        {
            "key": "dbname",
            "value": "create_delete_db"
        },
        {
            "key": "codebase",
            "value": "code"
        }
    ],
    "createdTimestamp": "2023-05-31 15:23:52.245000+00:00",
    "databases": [
        {
            "cacheC

AccessDeniedException: An error occurred (AccessDeniedException) when calling the GetKxConnectionString operation: 

## Clean Up

In [21]:
# Cluster Deletion
# ------------------------------------------------------------
db_list = list_kx_databases(client, environmentId=ENV_ID)
db_list

db_pdf = pd.DataFrame.from_dict(db_list).style.hide(axis='index')
display(db_pdf)
print("")

cluster_deleted=False

if DELETE_CLUSTER:   
    # list all clusters
    resp=client.get_kx_cluster(environmentId=ENV_ID, clusterName=CLUSTER_NAME)
    
    if resp['ResponseMetadata']['HTTPStatusCode'] != 200:
        sys.stderr.write("Error:\n {resp}")
    else:
        resp.pop('ResponseMetadata', None)

    if resp['status'] != 'DELETING':
        try:
            resp = client.delete_kx_cluster(environmentId=ENV_ID, clusterName=CLUSTER_NAME)
            if resp['ResponseMetadata']['HTTPStatusCode'] != 200:
                sys.stderr.write("Error:\n {resp}")
            else:
                resp.pop('ResponseMetadata', None)
        except Exception as e: 
            sys.stderr.write(f"Error deleting cluster: {CLUSTER_NAME}\n{e}")
            cluster_deleted = False

    try:
        wait_for_cluster_status(client, environmentId=ENV_ID, clusterName=CLUSTER_NAME, status='DELETED', show_wait=True)
        print()
        print("** DONE **")

        cluster_deleted = True
    except client.exceptions.ResourceNotFoundException:
        cluster_deleted = True
else:
    print(f"DELETE_CLUSTER: {DELETE_CLUSTER}")

databaseName,createdTimestamp,lastModifiedTimestamp
TAQ_2021_2D,2023-05-30 22:54:17.321000+00:00,2023-05-30 22:54:17.945000+00:00
create_delete_db,2023-05-31 13:52:16.839000+00:00,2023-05-31 13:52:17.712000+00:00
TAQ_2021H1,2023-05-30 22:55:27.556000+00:00,2023-05-30 22:59:43.785000+00:00
welcomedb,2023-05-30 22:53:12.481000+00:00,2023-05-30 22:53:13.165000+00:00
basictickdb,2023-05-30 23:21:59.431000+00:00,2023-05-31 00:18:15.745000+00:00



Cluster: cluster_create_delete_db status is DELETING, total wait 0:00:00, waiting 30 sec ...
Cluster: cluster_create_delete_db status is DELETING, total wait 0:00:30, waiting 30 sec ...
Cluster: cluster_create_delete_db status is DELETING, total wait 0:01:00, waiting 30 sec ...
Cluster: cluster_create_delete_db status is DELETING, total wait 0:01:30, waiting 30 sec ...
Cluster: cluster_create_delete_db status is DELETING, total wait 0:02:00, waiting 30 sec ...
Cluster: cluster_create_delete_db status is DELETING, total wait 0:02:30, waiting 30 sec ...
Cluster: cluster_create_delete_db status is DELETING, total wait 0:03:00, waiting 30 sec ...
Cluster: cluster_create_delete_db status is DELETING, total wait 0:03:30, waiting 30 sec ...
Cluster: cluster_create_delete_db status is DELETING, total wait 0:04:00, waiting 30 sec ...
Cluster: cluster_create_delete_db status is DELETING, total wait 0:04:30, waiting 30 sec ...
Cluster: cluster_create_delete_db status is DELETING, total wait 0:05

In [22]:
# Database Deletion
# Requires cluster to have been deleted
if DELETE_DATABASE:
    if cluster_deleted:
        # if the database exists, delete it
        if has_database(client, environmentId=ENV_ID, databaseName=DB_NAME):
            try:
                resp = client.delete_kx_database(environmentId=ENV_ID, databaseName=DB_NAME)
                if resp['ResponseMetadata']['HTTPStatusCode'] != 200:
                    sys.stderr.write("Error:\n {resp}")
                else:
                    resp.pop('ResponseMetadata', None)

                resp
            except Exception as e: 
                sys.stderr.write(f"Error: \n{e}")
        else:
            print(f"Database already deleted: {DB_NAME} ")
    else:
        print(f"Cluster deleted? {cluster_deleted}, will not delete database if cluster not deleted")
else:
    print(f"DELETE_DATABASE: {DELETE_DATABASE}")

In [23]:
db_list = list_kx_databases(client, environmentId=ENV_ID)
db_list=sorted(db_list, key=lambda d: d['databaseName']) 

db_pdf = pd.DataFrame.from_dict(db_list).style.hide(axis='index')
display(db_pdf)

databaseName,createdTimestamp,lastModifiedTimestamp
TAQ_2021H1,2023-05-30 22:55:27.556000+00:00,2023-05-30 22:59:43.785000+00:00
TAQ_2021_2D,2023-05-30 22:54:17.321000+00:00,2023-05-30 22:54:17.945000+00:00
basictickdb,2023-05-30 23:21:59.431000+00:00,2023-05-31 00:18:15.745000+00:00
welcomedb,2023-05-30 22:53:12.481000+00:00,2023-05-30 22:53:13.165000+00:00


In [24]:
cdf = get_clusters(client, ENV_ID)

display(cdf)

print( f"Last Run: {datetime.datetime.now()}" )

clusterName,status,clusterType,capacityConfiguration,commandLineArguments,clusterDescription,lastModifiedTimestamp,createdTimestamp,databaseName,cacheConfigurations
GATEWAY_basictickdb_20230530,RUNNING,GATEWAY,"{'nodeType': 'kx.s.4xlarge', 'nodeCount': 1}","[{'key': 's', 'value': '8'}, {'key': 'dbdir', 'value': 'basictickdb'}, {'key': 'codedir', 'value': 'basictick'}, {'key': 'rdb_name', 'value': 'RDB_basictickdb_20230530'}, {'key': 'hdb_name', 'value': 'HDB_basictickdb_20230530'}]",Created with create_GW notebook,2023-05-31 00:29:39.774000+00:00,2023-05-31 00:15:38.003000+00:00,None,nan
HDB_TAQ_2021H1,RUNNING,HDB,"{'nodeType': 'kx.s.32xlarge', 'nodeCount': 2}","[{'key': 's', 'value': '32'}, {'key': 'dbname', 'value': 'TAQ_2021H1'}, {'key': 'codebase', 'value': 'taqcode'}]",Created with create_cluster_TAQ_H1 notebook,2023-05-31 10:40:45.063000+00:00,2023-05-31 10:19:39.315000+00:00,TAQ_2021H1,"[{'cacheType': 'CACHE_1000', 'dbPaths': ['/2021.01.04/', '/2021.01.05/', '/2021.01.06/', '/2021.01.07/', '/2021.01.08/', '/2021.01.11/', '/2021.01.12/', '/2021.01.13/', '/2021.01.14/', '/2021.01.15/', '/2021.01.19/', '/2021.01.20/', '/2021.01.21/', '/2021.01.22/', '/2021.01.25/', '/2021.01.26/', '/2021.01.27/', '/2021.01.28/', '/2021.01.29/', '/2021.02.01/', '/2021.02.02/', '/2021.02.03/', '/2021.02.04/', '/2021.02.05/']}]"
HDB_TAQ_2021_2D,RUNNING,HDB,"{'nodeType': 'kx.s.32xlarge', 'nodeCount': 2}","[{'key': 's', 'value': '8'}, {'key': 'dbname', 'value': 'TAQ_2021_2D'}, {'key': 'codebase', 'value': 'taqcode'}]",Created with create_cluster_TAQ_2D notebook,2023-05-31 10:40:17.015000+00:00,2023-05-31 10:19:34.957000+00:00,TAQ_2021_2D,"[{'cacheType': 'CACHE_1000', 'dbPaths': ['/2021.01.04/', '/2021.01.05/']}]"
HDB_basictickdb_20230530,RUNNING,HDB,"{'nodeType': 'kx.s.xlarge', 'nodeCount': 3}","[{'key': 's', 'value': '8'}, {'key': 'dbdir', 'value': 'basictickdb'}, {'key': 'codedir', 'value': 'basictick'}]",Created with create_HDB for basic_tick notebook,2023-05-31 00:44:28.283000+00:00,2023-05-30 23:52:04.592000+00:00,basictickdb,"[{'cacheType': 'CACHE_1000', 'dbPaths': ['/']}]"
HDB_welcomedb,RUNNING,HDB,"{'nodeType': 'kx.s.2xlarge', 'nodeCount': 3}","[{'key': 's', 'value': '4'}, {'key': 'dbname', 'value': 'welcomedb'}, {'key': 'codebase', 'value': 'code'}]",Created with create_cluster_HDB notebook,2023-05-30 23:13:20.948000+00:00,2023-05-30 22:56:11.773000+00:00,welcomedb,"[{'cacheType': 'CACHE_1000', 'dbPaths': ['/']}]"
RDB_basictickdb_20230530,RUNNING,RDB,"{'nodeType': 'kx.s.2xlarge', 'nodeCount': 1}","[{'key': 's', 'value': '8'}, {'key': 'dbdir', 'value': 'basictickdb'}, {'key': 'codedir', 'value': 'basictick'}, {'key': 'tphostfile', 'value': 'tickerplant2'}]",Created with create_RDB notebook,2023-05-31 00:13:16.692000+00:00,2023-05-30 23:59:27.832000+00:00,basictickdb,[]
RDB_welcomedb,RUNNING,RDB,"{'nodeType': 'kx.s.2xlarge', 'nodeCount': 1}","[{'key': 's', 'value': '8'}, {'key': 'dbname', 'value': 'welcomedb'}, {'key': 'codebase', 'value': 'code'}]",Created with create_cluster_RDB notebook,2023-05-30 23:10:28.477000+00:00,2023-05-30 22:56:52.812000+00:00,welcomedb,[]
cluster_welcomedb,RUNNING,HDB,"{'nodeType': 'kx.s.xlarge', 'nodeCount': 3}","[{'key': 's', 'value': '4'}, {'key': 'dbname', 'value': 'welcomedb'}, {'key': 'codebase', 'value': 'welcomedb'}]",Demo Cluster for database welcomedb,2023-05-30 23:10:58.551000+00:00,2023-05-30 22:53:31.356000+00:00,welcomedb,"[{'cacheType': 'CACHE_1000', 'dbPaths': ['/']}]"


Last Run: 2023-05-31 14:49:42.394627
